In [134]:
from bs4 import BeautifulSoup as bs
import requests
import pprint as pp
import pandas as pd

main_link = 'https://hh.ru/search/vacancy'

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
          'Accept':'*/*'}

vaks = []

def page_worker(href):
    
    response = requests.get(href, headers=header)
    soup = bs(response.text, 'html.parser') 

    vak_block = soup.find('div', {'class':'vacancy-serp'})
    vak_list = vak_block.find_all('div', {'class':'vacancy-serp-item'}) 


    for vak in vak_list:
        vak_data = {}
        vak_name = vak.find('a', {'class':'bloko-link HH-LinkModifier'}) 
        vak_sum = vak.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'}) 
        if (vak_sum): 
            vak_sum = vak_sum.getText()   
        else:         
            vak_sum = ''
            
        vak_ref = vak_name['href']  
    
    
        vak_data['vak_name'] = vak_name.getText()
        #vak_data['vak_sum'] = vak_sum

        vak_sum_txt1 = vak_sum.replace('от', '', 1).replace('до', '', 1).replace('руб.', '', 1).replace(' ', '', 1)
        vak_sum_txt2 = re.split("-", vak_sum_txt1)
   
        if vak_sum.find("от") != -1:
            vak_data['vak_sum_min'] = vak_sum_txt2[0]          
            vak_data['vak_sum_max'] = None
        elif vak_sum.find("до") != -1:         
            vak_data['vak_sum_min'] = None
            vak_data['vak_sum_max'] = vak_sum_txt2[0]          
        elif vak_sum.find("-") != -1:
            vak_data['vak_sum_min'] = vak_sum_txt2[0]                  
            vak_data['vak_sum_max'] = vak_sum_txt2[1]                 
    
        vak_data['vak_sum_val'] = vak_sum[-4:]    
        
        vak_data['vak_ref'] = vak_ref
        vak_data['vak_source'] = 'hh.ru'    
        vaks.append(vak_data)
    




vaks_pages = soup.find_all('a', {'class':'bloko-button HH-Pager-Control'}) 
vaks_pages_count = len(vaks_pages)

i = 0
while i < vaks_pages_count:
    page_worker('https://hh.ru/'+vak_page[i]['href'])
    i = i + 1


df = pd.DataFrame(vaks) 
df[(df['vak_sum_val'] == 'руб.')]
    
    


,vak_name,vak_sum_val,vak_ref,vak_source,vak_sum_min,vak_sum_max
4,Диджитал-аналитик (Digital Analyst),руб.,https://hh.ru/vacancy/37707788?query=Data%20sc...,hh.ru,75 000,150 000
19,Ведущий инженер по тестированию,руб.,https://hh.ru/vacancy/36057559?query=Data%20sc...,hh.ru,None,250 000
23,Senior Data Engineer / Ведущий инженер данных,руб.,https://hh.ru/vacancy/37167337?query=Data%20sc...,hh.ru,None,290 000
48,Директор по инвестициям (Family office),руб.,https://hh.ru/vacancy/37506599?query=Data%20sc...,hh.ru,500 000,1 000 000
50,Senior Python Systems Engineer,руб.,https://hh.ru/vacancy/37618147?query=Data%20sc...,hh.ru,200 000,280 000
68,IT рекрутер / ресерчер (удаленно),руб.,https://hh.ru/vacancy/37541942?query=Data%20sc...,hh.ru,50 000,100 000
86,Researcher,руб.,https://hh.ru/vacancy/37504917?query=Data%20sc...,hh.ru,30 000,70 000
102,Аналитик данных (data analyst)/data scientist ...,руб.,https://hh.ru/vacancy/37221045?query=Data%20sc...,hh.ru,150 000,None
107,Data Scientist,руб.,https://hh.ru/vacancy/37424812?query=Data%20sc...,hh.ru,120 000,150 000
125,Head of I.T. recruitment practice,руб.,https://hh.ru/vacancy/37354140?query=Data%20sc...,hh.ru,50 000,500 000
